In [ ]:
import os
import shutil
import yaml
import time
import json
import cv2
import random
import pydicom
from PIL import Image
from pydicom.pixel_data_handlers.util import apply_voi_lut
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch
from collections import Counter
from ensemble_boxes import *
import copy
import os.path as osp
import mmcv
import numpy as np
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
KAGGLE = False
TEST = True
VER = 'v1'
if KAGGLE:
    DATA_PATH = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection'
    MDLS_PATH = f'/kaggle/input/vinbigdata-mmdet-models-{VER}'
    CACHE_PATH = '/kaggle/working/cache'
    SUBM_DIR = '/kaggle/working'
else:
    DATA_PATH = '/u01/mrorange/vinbigdata/data'
    WRK_DIR = f'{DATA_PATH}/workmmd'
    MDLS_PATH = f'/u01/mrorange/vinbigdata/models_mmdet_{VER}'
TEST_PATH = f'{DATA_PATH}/test'
TH = .4
with open(f'{MDLS_PATH}/params.json') as file:
    params = json.load(file)
print('loaded params:', params)
SIZE = params['img_size']

start_time = time.time()

In [ ]:
checkpoint = f'{MDLS_PATH}/epoch_8.pth'
cfg = f'{MDLS_PATH}/init_config.py'
model = init_detector(cfg, checkpoint, device='cuda:0')

In [ ]:
def read_xray(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array        
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8) 
    return data

In [ ]:
imgs_list = os.listdir(TEST_PATH)[:20] if TEST else os.listdir(TEST_PATH)
imgs_ids = []
pred_strings = []
for img_name in tqdm(imgs_list):
    img_id = img_name.split('.')[0]
    path = f'{TEST_PATH}/{img_name}'
    xray = read_xray(path)
    img = cv2.cvtColor(xray, cv2.COLOR_GRAY2RGB)
    result = inference_detector(model, img)
    if False:
        show_result_pyplot(model, img, result, score_thr=.25)
    bboxes = []
    for i, rt in enumerate(result):
        for ri in rt:
            if ri[4] >= TH:
                bboxes.append(
                    ' '.join([
                        str(i),
                        str(np.round(ri[4], 1)),
                        ' '.join([str(int(x)) for x in ri[:4]])
                    ])
                )
    imgs_ids.append(img_id)
    pred_strings.append(' '.join(bboxes) if len(bboxes) > 0 else '14 1 0 0 1 1')

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
subm_df = pd.DataFrame({'image_id': imgs_ids, 'PredictionString': pred_strings})
subm_df.to_csv('submission.csv', index=False)
print('submission dataframe saved:', subm_df.shape)
subm_df.tail()